### [W-okada's Deiteris' Fork Realtime Voice Changer](https://github.com/deiteris/voice-changer) | **Lightning.AI**

---

## **READ ME - VERY IMPORTANT**

You can use the following settings for optimal results:

Best performance (with good quality): `f0: fcpe | Chunk: 80.0ms or higher | Extra: 5s`<br>
Best quality: `f0: rmvpe | Chunk: 112.0ms or higher | Extra: 5s`<br>
**Don't forget to select your Lightning.AI GPU in the GPU field*

You can tune `Chunk` for lower/higher delay and `Extra` for better quality.

<br><br>

---

### Always use Lightning.AI GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Lightning.AI GPU so the Voice Changer can work faster and better
Use the lateral right menu, and click on **Studio Environment (the Processor Icon)** » **Switch To GPU** »  select a GPU, **check [pricing](https://lightning.ai/pricing)** for more info about how many hours you can use each GPUs and which for free.

---

# **Credits and Support**
Original W-okada Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Original instructions by [Hina](https://github.com/HinaBl)<br>
Wokada Deiteris Fork Realtime Voice Changer by [deiteris](https://github.com/deiteris)<br>
Tunnels (Port Viewer, Cloudflare, LocalTunnel, Horizon), Decryption and Lightning.AI Port by [Nick088](https://linktr.ee/Nick088)<br>

Need help? [AI Hub Discord](https://discord.gg/aihub)

---

# Clone repository and install dependencies
This first step will download the latest version of Wokada Deiteris Fork and install the dependencies. **It can take some time to complete.**

## (Optional) ClearConsole:
You can set `ClearConsole` to True or False wether you want to clear the output when the server is ready or not.

`Default: True`

In [ ]:
import os
import time
from IPython.display import clear_output

%cd /teamspace/studios/this_studio

!pip install colorama python-dotenv

from colorama import Fore, Style
import requests


LATEST_RELEASE = "https://api.github.com/repos/deiteris/voice-changer/releases/latest"
LINUX_PREBUILT = "voice-changer-linux-amd64-cuda.tar.gz"


print(f"{Fore.CYAN}> Downloading prebuilt executable...{Style.RESET_ALL}")

res = requests.get(LATEST_RELEASE)
release_info = res.json()

for asset in release_info['assets']:
   if not asset['name'].startswith(LINUX_PREBUILT):
      continue
   download_url = asset['browser_download_url']
   !wget -q --show-progress {download_url}

print(f"{Fore.GREEN}> Unpacking...{Style.RESET_ALL}")
!cat {LINUX_PREBUILT}.* | tar -xzf -
print(f"{Fore.GREEN}> Finished unpacking!{Style.RESET_ALL}")
!rm -rf {LINUX_PREBUILT}.*


%cd /teamspace/studios/this_studio

print(f"{Fore.GREEN}> Successfully downloaded and unpacked the binary!{Style.RESET_ALL}")

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!sudo apt-get update -y
!sudo apt-get -y install libportaudio2

if ClearConsole:
    clear_output()

print("Installed!")

# Set server configuration
This cell will set the server configuration.

In [ ]:
%cd /teamspace/studios/this_studio/MMVCServerSIO

from dotenv import set_key

set_key('.env', "SAMPLE_MODE", "")

print("Server successfully configurated!")

# Start Server **using Tunnels**
This cell will start the server via Tunnels, the first time that you run it will download the models, so it can take a few minutes (usually ~1-2 minutes)

## Tunnels

Select he type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one:


 A) Port Viewer: Select it in Tunnel, Click the + at the bottom of the right tab, click on Web Apps and install Port Viewer, run the cell, then wait for the Local URL to appear and click on Port Viewer on the right tab, Add a new port and write 18888 as the Port Number and optionally give it a name, after it's added, double click on the port name to see the UI, and to see it bigger you can click Open.

 B) Ngrok: Select it in Tunnel, get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new, run the cell, wait for the Local URL to appear and click on the Ngrok Tunnel Public URL which is above.

 C) Cloudflare: Select it in Tunnel, run the cell, wait for the Local URL to appear and click on the Cloudflare Tunnel Public URL which is above.

 D) LocalTunnel: Select it in Tunnel, run the cell, wait for the Local URL to appear, copy the LocalTunnel Password displayed under the public link and paste it in Tunnel Password of the LocalTunnel Tunnel Public URL which is above.

 E) Horizon: Select it in Tunnel, get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an hrzn login YOURID, you need to copy that id and put it in horizonid. Then run the cell, you will get an 'HORIZON: Authorize at https://hrzn.run/dashboard/settings/cli-token-requests/YOUR_ID', click it, do Approve. At the end, run the cell, wait for the Local URL to appear and click on the Horizon Tunnel Public URL which is above.


 ## (Optional) Ngrok Region
If you're using the Ngrok Tunnel, you can change to a region near to you or keep at United States if increase latency.

`Default Region: ap - Asia/Pacific (Singapore)`

## (Optional) ClearConsole:
You can set `ClearConsole` to True or False wether you want to clear the output when the server is ready or not.

`Default: True`

In [ ]:
#======================Tunnels===========================

Tunnel = "Port Viewer" #@param ["Port Viewer", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

Token = 'Ngrok | Horizon TOKEN' # @param {type:"string"}
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

ClearConsole = True  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

PORT = 18888

import json
import threading, time, socket
from IPython.display import clear_output
from dotenv import set_key

%cd /teamspace/studios/this_studio/

if Tunnel == "Port Viewer":
    print("Be sure to run the manual steps in the guide above for Port Viewer!")
    import urllib
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
  
    # Construct the origin URL from the endpoint IP 
    ip_origin = f"http://{endpoint_ip}:{PORT}"
  
    # The 'public_url' variable will hold the user-facing message.
    public_url = "Please use the Port Viewer URL from the Lightning AI UI explained in the guide above."
elif Tunnel == "Ngrok":
    !pip install pyngrok --quiet
    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);


    from pyngrok import ngrok

    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url
elif Tunnel == "Cloudflare":
    # download cloudflare
    !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !sudo dpkg -i cloudflared-linux-amd64.deb
    # Run cloudflare
    import time
    import subprocess

    # Define the command as a list of arguments
    command = ["cloudflared", "tunnel", "--url", "localhost:18888"]
    
    # Open the output file
    with open("nohup.out", "w") as f:
        # Start the cloudflared process in the background using Popen
        # stdout and stderr are redirected to the file 'f'
        proc = subprocess.Popen(command, stdout=f, stderr=subprocess.STDOUT)

    time.sleep(7)
    # Find and print the Cloudflare URL with a prefix
    cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
    if cloudflare_url:
        public_url = cloudflare_url[0] # Assign the first element (the string)
elif Tunnel == "LocalTunnel":
    # install
    # Update package manager lists and install curl (used to download the script)
    !sudo apt-get update -y
    !sudo apt-get install -y curl

    # Download and run the NodeSource setup script for Node.js v18 (a stable LTS version).
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - 

    # Install the modern 'nodejs' package. This also includes npm.
    !sudo apt-get install -y nodejs
    print("Node.js & npm installation complete!")
        
    !sudo npm install -g localtunnel
    import time
    import urllib
    # run localtunnel
    with open('url.txt', 'w') as file:
          file.write('')

    get_ipython().system_raw('lt --port 18888 >> url.txt 2>&1 &')

    time.sleep(4)

    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        public_url = file.read()
        public_url = public_url.replace("your url is: ", "")

elif Tunnel == "Horizon":
    # install
    import os
    # Update package manager lists and install curl (used to download the script)
    !sudo apt-get update -y
    !sudo apt-get install -y curl

    # Download and run the NodeSource setup script for Node.js v18 (a stable LTS version).
    !curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - 

    # Install the modern 'nodejs' package. This also includes npm.
    !sudo apt-get install -y nodejs
    print("Node.js & npm installation complete!")
    
    !sudo npm install -g @hrzn/cli
    !hrzn login $Token
    os.system(f"hrzn tunnel http://localhost:{PORT} >> url.txt 2>&1 &")
    time.sleep(5)

    with open('url.txt', 'r') as file:
      public_url = file.read()
      public_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      public_url = public_url[0]

%cd /teamspace/studios/this_studio/MMVCServerSIO

# Set the allowed origins based on the tunnel type
if Tunnel == "Port Viewer":
    # For Port Viewer, allow the IP-based origin
    allowed_origins = [ip_origin]
else:
    # For all other tunnels, allow their specific public URL
    allowed_origins = [public_url]
  
set_key('.env', "ALLOWED_ORIGINS", json.dumps(allowed_origins))


def wait_for_server():
    # This loop will run silently in the background.
    # Its only job is to wait until it can connect to the server port.
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            sock.close()
            break # Exit the loop once the server is ready

    # This code will only run AFTER the loop above breaks.
    if ClearConsole:
        clear_output()

    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    if Tunnel == "LocalTunnel":
        print("---------------------------------")        
        print(f"Local Tunnel Password: {endpoint_ip}")
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()

!./MMVCServerSIO

# Tunnels clean up
if Tunnel == "Ngrok":
    ngrok.disconnect(ngrokConnection.public_url)
elif Tunnel == "Cloudflare":
    !rm -rf nohup.out
elif Tunnel == "LocalTunnel":
    !rm -rf url.txt
elif Tunnel == "Horizon":
    !rm -rf url.txt
    !fuser -k ${PORT}

print("--------- SERVER STOPPED! ---------")

# Delete everything

Lightning.ai sessions storage persists, so if there is any Wokada Deiteris Fork updates, you need to delete everything first and then install it.

In [ ]:
!rm -rf /teamspace/studios/this_studio/MMVCServerSIO
!rm -rf /teamspace/studios/this_studio/nohup.out
!rm -rf /teamspace/studios/this_studio/url.txt